# Gradient of the Likelihood Rough Work

The main goal here is to lay out the framework for computing the gradient of the likelihood using the zeropadded framework I've been working on so far.

As a side note, could be a useful to write a small class that helps with simulating paramters needed to feed throgh things like the upcoming gradient code and the covariance inverse etc...

In [7]:
import os
import sys
from corrcal import SparseCov

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
target_dir = os.path.join(parent_dir, 'corrcal_gpu_pipeline', 'pipeline')
sys.path.insert(0, target_dir)

In [8]:
from zp_puregpu_funcs_py import *
import numpy as np
import cupy as cp
import corrcal

In [35]:
print(np.arange(5))

[0 1 2 3 4]


In [42]:
arr = np.arange(18)
arr = arr.reshape(6,3)
print(arr)
print(np.sum(arr, axis=1))

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]]
[ 3 12 21 30 39 48]


In [57]:
arr2 = np.arange(16)
arr3 = arr2.reshape(8,2)
arr2 = arr2.reshape(4, 2, 2)
print(arr2)
print(arr3)
print(np.sum(arr3, axis=1))
arr4 = np.sum(arr2, axis=2)
print(arr4)
print(arr4.reshape(2*4))

[[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]

 [[12 13]
  [14 15]]]
[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]]
[ 1  5  9 13 17 21 25 29]
[[ 1  5]
 [ 9 13]
 [17 21]
 [25 29]]
[ 1  5  9 13 17 21 25 29]


In [53]:
print(arr2**2)
print(arr3**2)

[[[  0   1]
  [  4   9]]

 [[ 16  25]
  [ 36  49]]

 [[ 64  81]
  [100 121]]

 [[144 169]
  [196 225]]]
[[  0   1]
 [  4   9]
 [ 16  25]
 [ 36  49]
 [ 64  81]
 [100 121]
 [144 169]
 [196 225]]


In [61]:
array = cp.arange(10)
print(array//2)
print(type(cp.asnumpy(array)))

[0 0 1 1 2 2 3 3 4 4]
<class 'numpy.ndarray'>


Testing to see whether I can eaily compare the existing code to what I'm about to write for the gradient calcuation and it looks like it should be pretty straightforward...

In [19]:
noise = np.random.rand(10)
src = np.random.rand(10, 2)
diffuse = np.random.rand(10, 3)
data = np.random.rand(10,1)
edges = np.array([0, 2, 6, 10])

cov = SparseCov(noise,src, diffuse, edges, 3, isinv=False)

In [31]:
invc = cov.inv()
print(cov.noise)
print(invc.noise)

p = invc@data
print(p)

print(invc.diff_mat)
print(cov.diff_mat)
print(1/0.86)

[0.60183562 0.00521249 0.1144356  0.01229719 0.56802334 0.84614436
 0.73455493 0.79159902 0.95521365 0.2137562 ]
[  1.66158328 191.84703243   8.73853931  81.31939537   1.76049103
   1.18183143   1.36136858   1.26326583   1.04688621   4.67822684]
[[-0.12669745]
 [ 0.22193126]
 [-0.49095372]
 [ 0.15443552]
 [ 0.79130251]
 [-0.1667372 ]
 [ 0.7155757 ]
 [ 0.16295217]
 [ 0.18278741]
 [-0.65078683]]
[[ 1.55063368e-01 -6.46780624e-01  2.33298533e-01]
 [ 1.36683209e+01  1.77115815e+00  4.55123269e-01]
 [ 1.57669419e-01  2.57547558e+00 -3.73847150e-01]
 [ 8.93304731e+00 -8.39466114e-02  5.50371490e-01]
 [ 2.45338841e-02  1.82439529e-01 -1.04037334e-01]
 [ 4.14336518e-02 -9.15318504e-02  6.65181755e-02]
 [ 2.17235722e-01  2.65100984e-01  1.93632792e-01]
 [ 6.34273696e-01  1.36044746e-03  1.07038621e-01]
 [ 5.18802432e-01 -1.38200924e-01 -1.72024120e-01]
 [ 4.50887161e-01  1.74482324e+00  5.59074875e-01]]
[[0.86214971 0.22446397 0.85472876]
 [0.65819665 0.6805963  0.75664286]
 [0.15220258 0.98788

In [ ]:
#pseudo-ish code to follow...

"""
Reminder to make smaller unit tests allong the way here. For example, before testing everything including the accumulate gradient function, construct all the p, q, s, and t matrices and compare to Bobby's code. Then proceed to testing new accumulate grad function against the full routine...
"""

#full grad function
def grad_nll():
    """

    """

    #0.5 zeropad EVERYTHING....

    #1. apply gains to the source and diffuse matrices (ie. constructing the 'true' convariance)

    #2. compute inverse cov components

    #(I think) can roughly follow current python/C implimentation for the next few steps without much change

    #3. Now compute p = C^-1 @ data => Might want to construct my own __matmul__ function for this

    #4. compute q = (C - N) @ G.T @ p

    #5. compute s and t

    #6. compute the inverse power

    #7. accumulate gradient

        #nominally, k1, k2 indices are constructed from looping over ant_inds. This is problematic since
        # ant_inds now do not follow the zeropadding scheme.

        #SOLN: return a new ant_1_ind and ant_2_ind that correspond to the indices of the zeropadded matrices

        #7.1.

    return

In [34]:
edges = np.array([0, 3, 5, 12, 20])
ant_inds = np.arange(20)
zp_ant_inds, lb, nbs = zeroPad(ant_inds, edges=edges, return_inv=False)
zp_ant_inds[1:] = np.where(zp_ant_inds[1:] ==0, np.nan, zp_ant_inds[1:])
print(zp_ant_inds)
print(lb)
zp_ant_inds = zp_ant_inds.reshape(nbs, lb, 1)
print(zp_ant_inds)

[ 0.  1.  2. nan nan nan nan nan  3.  4. nan nan nan nan nan nan  5.  6.
  7.  8.  9. 10. 11. nan 12. 13. 14. 15. 16. 17. 18. 19.]
8
[[[ 0.]
  [ 1.]
  [ 2.]
  [nan]
  [nan]
  [nan]
  [nan]
  [nan]]

 [[ 3.]
  [ 4.]
  [nan]
  [nan]
  [nan]
  [nan]
  [nan]
  [nan]]

 [[ 5.]
  [ 6.]
  [ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]
  [nan]]

 [[12.]
  [13.]
  [14.]
  [15.]
  [16.]
  [17.]
  [18.]
  [19.]]]


In [32]:
arr = np.arange(10)
print(arr[np.nan])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices